# BentoML Example:  Deploy to Azure Container Instances

[BentoML](http://bentoml.ai) is an open source framework for building, shipping and running machine learning services. It provides high-level APIs for defining an ML service and packaging its artifacts, source code, dependencies, and configurations into a production-system-friendly format that is ready for deployment.

This notebook demonstrates how to use BentoML to deploy a machine learning model as a REST API endpoint to Microsoft Azure Container Instances. For this demo, we are using the [Sentiment Analysis with Scikit-learn](https://github.com/bentoml/BentoML/blob/master/examples/sklearn-sentiment-clf/sklearn-sentiment-clf.ipynb) example, using dataset from [Sentiment140](http://help.sentiment140.com/for-students/).

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=nb&ea=open&el=official-example&dt=deploy-with-google-cloudrun)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install -I bentoml
!pip install sklearn pandas numpy

In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

import bentoml

# Prepare Dataset

In [4]:
%%bash

if [ ! -f ./trainingandtestdata.zip ]; then
    wget -q http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
    unzip -n trainingandtestdata.zip
fi

Archive:  trainingandtestdata.zip
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv  


In [5]:
columns = ['polarity', 'tweetid', 'date', 'query_name', 'user', 'text']
dftrain = pd.read_csv('training.1600000.processed.noemoticon.csv',
                      header = None,
                      encoding ='ISO-8859-1')
dftest = pd.read_csv('testdata.manual.2009.06.14.csv',
                     header = None,
                     encoding ='ISO-8859-1')
dftrain.columns = columns
dftest.columns = columns

# Model Training

In [6]:
sentiment_lr = Pipeline([
                         ('count_vect', CountVectorizer(min_df = 100,
                                                        ngram_range = (1,1),
                                                        stop_words = 'english')), 
                         ('lr', LogisticRegression())])
sentiment_lr.fit(dftrain.text, dftrain.polarity)

/Users/hongjian/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('count_vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=100,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('lr',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='warn', n_jobs=None,
              

In [7]:
Xtest, ytest = dftest.text[dftest.polarity!=2], dftest.polarity[dftest.polarity!=2]
print(classification_report(ytest,sentiment_lr.predict(Xtest)))

              precision    recall  f1-score   support

           0       0.85      0.80      0.83       177
           4       0.82      0.86      0.84       182

    accuracy                           0.83       359
   macro avg       0.83      0.83      0.83       359
weighted avg       0.83      0.83      0.83       359



In [8]:
sentiment_lr.predict([Xtest[0]])

array([4])

#  BentoService for model serving

In [9]:
%%writefile sentiment_lr_model.py
import pandas as pd
import bentoml
from bentoml.artifact import PickleArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([PickleArtifact('sentiment_lr')])
@bentoml.env(pip_dependencies=["scikit-learn", "pandas"])
class SentimentLRModel(bentoml.BentoService):
    
    @bentoml.api(DataframeHandler, typ='series')
    def predict(self, series):
        """
        predict expects pandas.Series as input
        """        
        return self.artifacts.sentiment_lr.predict(series)

Writing sentiment_lr_model.py


# Save BentoService to file bundle

In [10]:
from sentiment_lr_model import SentimentLRModel

# Initialize bentoML model with artifacts

bento_model = SentimentLRModel()
bento_model.pack('sentiment_lr', sentiment_lr)

# Save bentoML model to directory
saved_path = bento_model.save()

# print the directory containing exported model archive (prefixed with model name and version)
print(saved_path)

[2020-01-20 13:56:20,996] INFO - BentoService bundle 'SentimentLRModel:20200120135559_A351E9' created at: /private/var/folders/dc/dtsln2wx0s3202znr340xfdr0000gn/T/bentoml-temp-evnkygx1
[2020-01-20 13:56:21,440] INFO - BentoService bundle 'SentimentLRModel:20200120135559_A351E9' created at: /Users/hongjian/bentoml/repository/SentimentLRModel/20200120135559_A351E9
/Users/hongjian/bentoml/repository/SentimentLRModel/20200120135559_A351E9


# Load BentoService from saved bundle

In [11]:
import bentoml

# Load exported bentoML model archive from path
bento_model = bentoml.load(saved_path)

# Call predict on the restored sklearn model
bento_model.predict(pd.Series(["hello", "hi"]))

[2020-01-20 13:58:03,170] WARNING - Module `sentiment_lr_model` already loaded, using existing imported module.


array([4, 4])

In [12]:
bento_tag = '{name}:{version}'.format(name=bento_model.name, version=bento_model.version)
print(bento_tag)

SentimentLRModel:20200120135559_A351E9


# Deploy BentoService with Azure Container Instances

This tutorial is modified from [Create and deploy container image to Azure Container Instances](https://docs.microsoft.com/en-us/azure/container-instances/container-instances-tutorial-deploy-app)


1. Install Azure CLI and docker
https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest

In [14]:
!az --version

azure-cli                         2.0.80

command-modules-nspkg              2.0.3
core                              2.0.80
nspkg                              3.0.4
telemetry                          1.0.4

Python location '/usr/local/Cellar/azure-cli/2.0.80/libexec/bin/python'
Extensions directory '/Users/hongjian/.azure/cliextensions'

Python (Darwin) 3.8.1 (default, Dec 27 2019, 18:06:00) 
[Clang 11.0.0 (clang-1100.0.33.16)]

Legal docs and information: aka.ms/AzureCliLegal



Your CLI is up-to-date.

Please let us know how we are doing: https://aka.ms/clihats


2. Create Azure container registry

In [15]:
! az login

You have logged in. Now let us find all the subscriptions to which you have access...
[
  {
    "cloudName": "AzureCloud",
    "id": "d3fe34fd-019d-47b0-a485-de3688e03bdd",
    "isDefault": true,
    "name": "Azure subscription 1",
    "state": "Enabled",
    "tenantId": "1f81e1a8-b059-4e1f-ab49-3ec3c0547d92",
    "user": {
      "name": "7lagrange@gmail.com",
      "type": "user"
    }
  }
]


In [16]:
# Create resource group
! az group create --name sentiment_azure --location eastus

{
  "id": "/subscriptions/d3fe34fd-019d-47b0-a485-de3688e03bdd/resourceGroups/sentiment_azure",
  "location": "eastus",
  "managedBy": null,
  "name": "sentiment_azure",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


In [17]:
# Create Azure container registry
! az acr create --resource-group sentiment_azure --name bentosentimentlrmodel --sku Basic --admin-enabled true

{
  "adminUserEnabled": true,
  "creationDate": "2020-01-20T22:04:09.741079+00:00",
  "id": "/subscriptions/d3fe34fd-019d-47b0-a485-de3688e03bdd/resourceGroups/sentiment_azure/providers/Microsoft.ContainerRegistry/registries/bentosentimentlrmodel",
  "location": "eastus",
  "loginServer": "bentosentimentlrmodel.azurecr.io",
  "name": "bentosentimentlrmodel",
  "networkRuleSet": null,
  "policies": {
    "quarantinePolicy": {
      "status": "disabled"
    },
    "retentionPolicy": {
      "days": 7,
      "lastUpdatedTime": "2020-01-20T22:04:11.344403+00:00",
      "status": "disabled"
    },
    "trustPolicy": {
      "status": "disabled",
      "type": "Notary"
    }
  },
  "provisioningState": "Succeeded",
  "resourceGroup": "sentiment_azure",
  "sku": {
    "name": "Basic",
    "tier": "Basic"
  },
  "status": null,
  "storageAccount": null,
  "tags": {},
  "type": "Microsoft.ContainerRegistry/registries"
}


In [18]:
# Log in to container registry
!az acr login --name bentosentimentlrmodel

Login Succeeded


In [19]:
!az acr show --name BentoSentimentLRModel --query loginServer --output table

Result
--------------------------------
bentosentimentlrmodel.azurecr.io


3. prepare BentoML docker image

In [39]:
print(saved_path)

/Users/hongjian/bentoml/repository/SentimentLRModel/20200120135559_A351E9


In [46]:
cd '/Users/hongjian/bentoml/repository/SentimentLRModel/20200120135559_A351E9'

/Users/hongjian/bentoml/repository/SentimentLRModel/20200120135559_A351E9


In [49]:
!docker build -t bentosentimentlrmodel.azurecr.io/sentimentlrmodel .

Sending build context to Docker daemon  8.314MB
Step 1/12 : FROM continuumio/miniconda3:4.7.12
 ---> 406f2b43ea59
Step 2/12 : ENTRYPOINT [ "/bin/bash", "-c" ]
 ---> Using cache
 ---> 26c44e044c6f
Step 3/12 : EXPOSE 5000
 ---> Using cache
 ---> 876689dac8b2
Step 4/12 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential      && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> b55b5cae6456
Step 5/12 : RUN conda install pip numpy scipy       && pip install gunicorn
 ---> Using cache
 ---> 773f610a13fe
Step 6/12 : COPY . /bento
 ---> b52183039970
Step 7/12 : WORKDIR /bento
 ---> Running in 57f7dba40805
Removing intermediate container 57f7dba40805
 ---> 682998f07f71
Step 8/12 : RUN conda env update -n base -f /bento/environment.yml
 ---> Running in 670b1c9eb3fe
Solving environment: ...working... done
#
# To activate this environment, use
#
#     $ conda activate base
#
# To deactivate an active environment, 

  Created wheel for tabulate: filename=tabulate-0.8.6-cp37-none-any.whl size=23274 sha256=6f9770763e9528af290b9b94da848649771241cab19b97b325b278d89f30f67c
  Stored in directory: /root/.cache/pip/wheels/9c/9b/f4/eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e
  Created wheel for alembic: filename=alembic-1.3.2-py2.py3-none-any.whl size=151128 sha256=f28d532c14cecf64ac44ee6c3b9f47862326c5ef670ad685e87f033a136f4e6a
  Stored in directory: /root/.cache/pip/wheels/5c/66/53/e0633382ac8625ab1c099db6a290d1b6b24f849a4666a57105
  Created wheel for python-json-logger: filename=python_json_logger-0.1.11-py2.py3-none-any.whl size=5076 sha256=0cde2d7daaefd14b20c09b437d588b947a706068c52144bb1090c9e754ee59c3
  Stored in directory: /root/.cache/pip/wheels/97/f7/a1/752e22bb30c1cfe38194ea0070a5c66e76ef4d06ad0c7dc401
  Created wheel for cerberus: filename=Cerberus-1.3.2-cp37-none-any.whl size=54336 sha256=8a67db2d0f91cf2772cd1a07eb7a20424fe5805a92ba7842876edfac65286dec
  Stored in directory: /root/.cach

In [50]:
!docker push bentosentimentlrmodel.azurecr.io/sentimentlrmodel

The push refers to repository [bentosentimentlrmodel.azurecr.io/sentimentlrmodel]

4358219f: Preparing 
6e8a3988: Preparing 
25e8c992: Preparing 
68afe3bd: Preparing 
1e1a7808: Preparing 
cb249b79: Preparing 
190fd43a: Preparing 


4358219f: Pushing  183.1MB/200.3MB7APushing  43.19MBPushing  66.68MB/150.5MBPushing  72.36MB/160.8MB

68afe3bd: Pushing    404MB/1.109GB

68afe3bd: Pushing  1.109GB/1.109GBPushing    526MB/1.109GB

68afe3bd: Pushed   1.115GBlatest: digest: sha256:8a680917935dd096c296147b722c2f2002b7e5c8c2a382db2161e9c64a752c74 size: 2012


In [51]:
!az acr repository list --name BentoSentimentLRModel --output table

Result
----------------
sentimentlrmodel


## Deploy the container image to Azure Container Instances
        

In [53]:
!az acr show --name BentoSentimentLRModel --query loginServer

"bentosentimentlrmodel.azurecr.io"


In [54]:
!az acr credential show -n BentoSentimentLRModel

{
  "passwords": [
    {
      "name": "password",
      "value": "+dqLfyU44bJmJTBxXckeDvanxDDTrcCU"
    },
    {
      "name": "password2",
      "value": "KZ7qsX5gvleMQT5jZ=BSoh+jam8l+nAO"
    }
  ],
  "username": "bentosentimentlrmodel"
}


In [56]:
# - Set Container Port from 5000
!az container create --resource-group sentiment_azure \
--name sentimentlrmodel \
--image bentosentimentlrmodel.azurecr.io/sentimentlrmodel \
--cpu 1 \
--memory 1 \
--registry-login-server bentosentimentlrmodel.azurecr.io \
--registry-username bentosentimentlrmodel \
--registry-password KZ7qsX5gvleMQT5jZ=BSoh+jam8l+nAO \
--dns-name-label bentosentimentlrmodel777 \
--ports 5000



{- Finished ..
  "containers": [
    {
      "command": null,
      "environmentVariables": [],
      "image": "bentosentimentlrmodel.azurecr.io/sentimentlrmodel",
      "instanceView": {
        "currentState": {
          "detailStatus": "",
          "exitCode": null,
          "finishTime": null,
          "startTime": "2020-01-20T22:43:38+00:00",
          "state": "Running"
        },
        "events": [
          {
            "count": 1,
            "firstTimestamp": "2020-01-20T22:41:32+00:00",
            "lastTimestamp": "2020-01-20T22:41:32+00:00",
            "message": "pulling image \"bentosentimentlrmodel.azurecr.io/sentimentlrmodel\"",
            "name": "Pulling",
            "type": "Normal"
          },
          {
            "count": 1,
            "firstTimestamp": "2020-01-20T22:43:35+00:00",
            "lastTimestamp": "2020-01-20T22:43:35+00:00",
            "message": "Successfully pulled image \"bentosentimentlrmodel.azurecr.io/sentimentlrmodel\"",
       

In [57]:
!az container show --resource-group sentiment_azure --name sentimentlrmodel --query instanceView.state

"Running"


In [58]:
!az container show --resource-group sentiment_azure --name sentimentlrmodel --query ipAddress.fqdn

"bentosentimentlrmodel777.eastus.azurecontainer.io"


In [59]:
!curl -X \
POST "http://bentosentimentlrmodel777.eastus.azurecontainer.io:5000/predict" \
--header "Content-Type: application/json" \
-d '["good movie", "bad food", "i feel happy today"]'

[4, 0, 4]

## Clean up deployed service on Azure Container Instances




In [60]:
!az group delete --name sentiment_azure

Are you sure you want to perform this operation? (y/n): ^C
